### Libraries Allowed
pandas, scipy, numpy
sklearn
datetime
dateutil
prophet
patsy (for model matrix), statsmodels
xgboost
warnings

In [1]:
import sys
import time
import pandas as pd
import numpy as np
import scipy
from scipy.stats.mstats import winsorize
import datetime
import dateutil
from tqdm import tqdm
import warnings

import statsmodels
import xgboost
import prophet
import patsy

from datetime import datetime, timedelta
import statsmodels.api as sm


Importing plotly failed. Interactive plots will not work.


In [ ]:
import matplotlib.pyplot as plt
import time

In [ ]:
"""
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.ensemble import GradientBoostingRegressor
import random
"""

In [ ]:
#warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.filterwarnings('ignore')

In [615]:
df1 = pd.DataFrame({'A': [1, 2, 3], 'B': [3, 4, 5]})
#df.columns[0:]

df2 = pd.DataFrame({'A': [1, 2], 'C': [3, 4]})

df3 = df1
df3.iloc[:, 1:] = (df1.iloc[:, :] + df2.iloc[:, :]).fillna(0)

df3

df1

,A,B
0,1,2.0
1,2,4.0
2,3,0.0


AttributeError: 'list' object has no attribute 'dtype'

In [ ]:
def process_data(train_file, test_file, test_with_label_file):
    # Fill in later
    return
    

In [ ]:
fold_count = 1
for i in tqdm(range(1, fold_count + 1)):
    train_file = f"Proj2_Data/fold_{i}/train.csv"
    test_file = f"Proj2_Data/fold_{i}/test.csv"
    test_with_label_file = f"Proj2_Data/test_with_label.csv"
    process_data(train_file, test_file, test_with_label_file)


In [ ]:
print(f"train_file={train_file}")
train_df = pd.read_csv(train_file)
train_df.head()

plt.

print(f"test_file={test_file}")
test_df = pd.read_csv(test_file)
test_df.head()

test_df.count()

In [ ]:
train_df

In [ ]:
pd.read_csv(f"Proj2_Data/test_with_label.csv")

In [ ]:
fig, ax = plt.subplots()  # Create a figure containing a single axes.
ax.plot([1, 2, 3, 4], [1, 4, 2, 3])  # Plot some data on the axes.

fig, axs = plt.subplots(2,2)  # Create a figure containing a single axes.
for i in range(2):
    for j in range(2):
        axs[i,j].plot([1, 2, 3, 4], [1, 4, 2, 3])  # Plot some data on the axes.
        print()

plt.subplot_mosaic([['left_top', 'right'],
                    ['left', 'right']])

#plt.plot([1, 2, 3, 4], [1, 4, 2, 3])

In [ ]:
plt.scatter([1, 2, 3, 4], [1, 4, 2, 3])


In [ ]:
train_df['Date'] = pd.to_datetime(train_df['Date'])

In [ ]:

train_df_tmp = train_df[1:60]
plt.plot(train_df_tmp['Date'], train_df_tmp['Weekly_Sales'])

In [ ]:
train_df_tmp.info()

In [ ]:
train_df.groupby(train_df['Date']).count()

In [ ]:
train_df.plot(subplots=True)

In [ ]:
train_df.describe()

In [ ]:
train_df.info()

In [ ]:
train_df.iloc[0:5, 0:3]

In [ ]:
train_df_new = train_df.set_index(['Date'])

In [ ]:
train_df_new.iloc[56:112].plot(subplots=True)

In [ ]:
plt.scatter(train_df['Date'], train_df['Weekly_Sales'])

In [ ]:
train_df_new.iloc[56:112]

In [520]:
def myeval(num_folds=10):
    file_path = 'Proj2_Data/test_with_label.csv'
    test_with_label = pd.read_csv(file_path)
    #num_folds = 10
    wae = []

    for i in range(num_folds):
        file_path = f'Proj2_Data/fold_{i+1}/test.csv'
        test = pd.read_csv(file_path)
        test = test.drop(columns=['IsHoliday']).merge(test_with_label, on=['Date', 'Store', 'Dept'])

        file_path = f'Proj2_Data/fold_{i+1}/mypred.csv'
        test_pred = pd.read_csv(file_path)

        # Left join with the test data
        new_test = test_pred.merge(test, on=['Date', 'Store', 'Dept'], how='left')

        # Compute the Weighted Absolute Error
        actuals = new_test['Weekly_Sales']
        preds = new_test['Weekly_Pred']
        weights = new_test['IsHoliday'].apply(lambda x: 5 if x else 1)
        wae.append(sum(weights * abs(actuals - preds)) / sum(weights))

    for value in wae:
        print(f"wae_by_fold={value:.3f}")
    print(f"overall wae={sum(wae) / len(wae):.3f}")
    
    # print(wae)
    # print(np.mean(wae))    
        
    return wae

### Approach#1

In [ ]:
num_folds = 1

for i in range(1, 11):

#i = 1
# Reading train data
    file_path = f'Proj2_Data/fold_{i}/train.csv'
    train = pd.read_csv(file_path)

    # Reading test data
    file_path = f'Proj2_Data/fold_{i}/test.csv'
    test = pd.read_csv(file_path)

    most_recent_date = train['Date'].max()

    tmp_train = train[train['Date'] == most_recent_date].copy()

    # Filter and select necessary columns
    tmp_train.rename(columns={'Weekly_Sales': 'Weekly_Pred'}, inplace=True)
    tmp_train = tmp_train.drop(columns=['Date', 'IsHoliday'])


    # Left join with the test data
    test_pred = test.merge(tmp_train, on=['Dept', 'Store'], how='left')

    #test_pred['Weekly_Pred'] = 0

    # Fill NaN values with 0 for the Weekly_Pred column
    test_pred['Weekly_Pred'].fillna(0, inplace=True)
    test_pred = test_pred.drop(columns=["IsHoliday"])

    # Write the output to CSV
    file_path = f'Proj2_Data/fold_{i}/mypred.csv'
    test_pred.to_csv(file_path, index=False)

#test_pred

In [ ]:
wae = myeval()
for value in wae:
    print(f"\t{value:.3f}")

print(f"{sum(wae) / len(wae):.3f}")

print(wae)
print(np.mean(wae))

### Approach - 2

In [ ]:
for i in range(1, 11):
    # Reading train data
    file_path = f'Proj2_Data/fold_{i}/train.csv'
    train = pd.read_csv(file_path)

    # Reading test data
    file_path = f'Proj2_Data/fold_{i}/test.csv'
    test = pd.read_csv(file_path)

    # Define start and end dates based on test data
    start_last_year = pd.to_datetime(test['Date'].min()) - timedelta(days=375)
    end_last_year = pd.to_datetime(test['Date'].max()) - timedelta(days=350)

    # Filter train data based on the defined dates and compute 'Wk' column
    tmp_train = train[(train['Date'] > str(start_last_year)) 
                      & (train['Date'] < str(end_last_year))].copy()
    tmp_train['Date'] = pd.to_datetime(tmp_train['Date'])  
    tmp_train['Wk'] = tmp_train['Date'].dt.isocalendar().week
    tmp_train.rename(columns={'Weekly_Sales': 'Weekly_Pred'}, inplace=True)
    tmp_train.drop(columns=['Date', 'IsHoliday'], inplace=True)

    # Compute 'Wk' column for test data
    test['Date'] = pd.to_datetime(test['Date'])
    test['Wk'] = test['Date'].dt.isocalendar().week

    # Left join with the tmp_train data
    test_pred = test.merge(tmp_train, on=['Dept', 'Store', 'Wk'], how='left').drop(columns=['Wk'])

    # Fill NaN values with 0 for the Weekly_Pred column
    test_pred['Weekly_Pred'].fillna(0, inplace=True)
    test_pred = test_pred.drop(columns=["IsHoliday"])

    # Save the output to CSV
    file_path = f'Proj2_Data/fold_{i}/mypred.csv'
    test_pred.to_csv(file_path, index=False)

In [ ]:
wae = myeval()
for value in wae:
    print(f"\t{value:.3f}")
print(f"{sum(wae) / len(wae):.3f}")

print(wae)
print(np.mean(wae))

### Approach - 3

In [590]:
def preprocess(data):
    tmp = pd.to_datetime(data['Date'])
    data['Wk'] = tmp.dt.isocalendar().week
    data['Yr'] = tmp.dt.year
    data['Wk'] = pd.Categorical(data['Wk'], categories=[i for i in range(1, 53)])  # 52 weeks 
#    data['IsHoliday'] = data['IsHoliday'].apply(int)
    return data


In [591]:
%%time
# The code is for fold number =2

foldnum = 2

#for foldnum in tqdm(range(1, 11)):
for foldnum in tqdm([1]):
    # Reading train data
    file_path = f'Proj2_Data/fold_{foldnum}/train.csv'
    train = pd.read_csv(file_path)

    # Reading test data
    file_path = f'Proj2_Data/fold_{foldnum}/test.csv'
    test = pd.read_csv(file_path)

    # pre-allocate a pd to store the predictions
    test_pred = pd.DataFrame()

    train_pairs = train[['Store', 'Dept']].drop_duplicates(ignore_index=True)
    test_pairs = test[['Store', 'Dept']].drop_duplicates(ignore_index=True)
    unique_pairs = pd.merge(train_pairs, test_pairs, how = 'inner', on =['Store', 'Dept'])

    train_split = unique_pairs.merge(train, on=['Store', 'Dept'], how='left')
    train_split = preprocess(train_split)
    y, X = patsy.dmatrices('Weekly_Sales ~ Weekly_Sales + Store + Dept + Yr  + Wk', 
                           data = train_split, 
                           return_type='dataframe')
    train_split = dict(tuple(X.groupby(['Store', 'Dept'])))


    test_split = unique_pairs.merge(test, on=['Store', 'Dept'], how='left')
    test_split = preprocess(test_split)
    y, X = patsy.dmatrices('Yr ~ Store + Dept + Yr  + Wk', 
                           data = test_split, 
                           return_type='dataframe')
    X['Date'] = test_split['Date']
    test_split = dict(tuple(X.groupby(['Store', 'Dept'])))

    keys = list(train_split)

    for key in keys:
        X_train = train_split[key]
        X_test = test_split[key]

        Y = X_train['Weekly_Sales']
        X_train = X_train.drop(['Weekly_Sales','Store', 'Dept'], axis=1)

        cols_to_drop = X_train.columns[(X_train == 0).all()]
        X_train = X_train.drop(columns=cols_to_drop)
        X_test = X_test.drop(columns=cols_to_drop)

        cols_to_drop = []
        for i in range(len(X_train.columns) - 1, 1, -1):  # Start from the last column and move backward
            col_name = X_train.columns[i]
            # Extract the current column and all previous columns
            tmp_Y = X_train.iloc[:, i].values
            tmp_X = X_train.iloc[:, :i].values

            coefficients, residuals, rank, s = np.linalg.lstsq(tmp_X, tmp_Y, rcond=None)
            if np.sum(residuals) < 1e-10:
                    cols_to_drop.append(col_name)

        X_train = X_train.drop(columns=cols_to_drop)
        X_test = X_test.drop(columns=cols_to_drop)

        model = sm.OLS(Y, X_train).fit()
        mycoef = model.params.fillna(0)

        tmp_pred = X_test[['Store', 'Dept', 'Date']]
        X_test = X_test.drop(['Store', 'Dept', 'Date'], axis=1)

        tmp_pred['Weekly_Pred'] = np.dot(X_test, mycoef)
        test_pred = pd.concat([test_pred, tmp_pred], ignore_index=True)

    test_pred['Weekly_Pred'].fillna(0, inplace=True)

    file_path = f'Proj2_Data/fold_{foldnum}/mypred.csv'
    print(file_path)
    test_pred.to_csv(file_path, index=False)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:26<00:00, 26.17s/it]

Proj2_Data/fold_1/mypred.csv
CPU times: user 2min 15s, sys: 1min 24s, total: 3min 39s
Wall time: 26.2 s


In [592]:
%%time

wae = myeval(num_folds=1)


wae_by_fold=2049.347
overall wae=2049.347
CPU times: user 341 ms, sys: 377 ms, total: 717 ms
Wall time: 93.9 ms


### Approach - 4

In [573]:
def preprocess(data):
        tmp = pd.to_datetime(data["Date"])
        data["Wk"] = tmp.dt.isocalendar().week
        data["Yr"] = tmp.dt.year
        data["Wk"] = pd.Categorical(data["Wk"], categories=[i for i in range(1, 53)])  # 52 weeks
        # data["IsHoliday"] = data["IsHoliday"].apply(int)
        return data

def pca_smooth_train_v1(train):
    """
    https://campuswire.com/c/G06C55090/feed/737

    Your description sounds right: Smooth train one dept at a time, combine each smoothed store data to create the whole smoothed train.
    Make sure you use this smooth dataframe in subsequent steps. Otherwise, the errors will be very high.

    Also, make sure you subtract the mean correctly. There are two ways:  get the mean by store and dept or just by store as outlined in
    the Professor's post. I ended up using (Store, Dept). You can actually run the whole process without subtracting the mean and once all
    the other logic is working correctly  you can add this process to see how it impacts the results. Divide and conquer. In my case,
    I got 1,583 without subtracting the mean before SVD, which is slightly better compared to 1,589.

    Regarding SVD, I used k=8 except when there are fewer components. In that case,  I am using S.shape.

    Last, when multiplying the matrices I am using: U @ S @ Vt without forcing (S @ Vt) first. It seems some people got a different result.
    I am using Windows and Python 3.11 and that differ in other platforms.
    """

    smooth_dept_trains = []
    departments = train['Dept'].unique()

#     print(f"train=\n {train}")
#     print(f"departments=\n {departments} {len(departments)}")
#     print(f"train['Store'].unique()=\n {train['Store'].unique()} {len(train['Store'].unique())}")
    
    for department in departments:
        # Filter rows where Dept is equal to 1
        filtered_train = train[train['Dept'] == department]
        # Select only the columns 'Store', 'Date', and 'Weekly_Sales'
        selected_columns = filtered_train[['Store', 'Date', 'Weekly_Sales']]
        # Pivot table to spread 'Store' values into columns, with 'Weekly_Sales' as values
        train_dept_ts = selected_columns.pivot(index='Date', columns='Store', values='Weekly_Sales').reset_index()
        # Smooth department data
        X_train = train_dept_ts.iloc[:, 1:]
        #print(f"train_dept_ts = \n  {train_dept_ts}")
        
#         print(f"filtered_train=\n {filtered_train}")
#         print(f"train_dept_ts=\n {train_dept_ts}")
        

        # store_means = X_train.mean(axis=1).to_list()
        # X_train = X_train.sub(store_means, axis=0)

        X_train = X_train.to_numpy()
        store_means = np.nanmean(X_train, axis=0)
        inds = np.where(np.isnan(X_train))
        X_train[inds] = np.take(store_means, inds[1])

        try:
            U, D, V_t = np.linalg.svd(X_train, full_matrices=False)
            D[8:] = 0
            F_train = U @ np.diag(D) @ V_t
            F_train = pd.DataFrame(F_train, columns=[str(x) for x in range(1, F_train.shape[1]+1)])

            # F_train = F_train.add(store_means, axis=0)

            F_train["Date"] = train_dept_ts["Date"]
            smooth_dept_train = pd.melt(F_train, id_vars=['Date'], var_name=['Store'], value_name='Weekly_Sales')
            smooth_dept_train["Dept"] = department

        except np.linalg.LinAlgError:
            smooth_dept_train = selected_columns

        smooth_dept_train["Store"] = smooth_dept_train["Store"].astype(np.int64)
        smooth_dept_trains.append(smooth_dept_train)

    smooth_train = pd.concat(smooth_dept_trains, ignore_index=True)
    return smooth_train

def pca_smooth_train_v2(train):
    smooth_dept_trains = []
    departments = train['Dept'].unique()

    for department in departments:
        # Filter rows where Dept is equal to 1
        filtered_train = train[train['Dept'] == department]
        # Select only the columns 'Store', 'Date', and 'Weekly_Sales'
        selected_columns = filtered_train[['Store', 'Date', 'Weekly_Sales']]
        # Pivot table to spread 'Store' values into columns, with 'Weekly_Sales' as values
        train_dept_ts = selected_columns.pivot(index='Date', columns='Store', values='Weekly_Sales').reset_index()

        X_train = train_dept_ts.iloc[:, 1:]

        # Smooth department data
        X_train = X_train.to_numpy()
        X_train = np.nan_to_num(X_train)
        store_means = np.mean(X_train, axis=0)
        X_train = X_train - store_means
        X_train = np.transpose(X_train)

        U, D, V_t = np.linalg.svd(X_train, full_matrices=False)
        D[8:] = 0
        F_train = U @ np.diag(D) @ V_t

        stores_list = train_dept_ts.columns[1:]
        F_train = pd.DataFrame(np.transpose(F_train), columns=stores_list)
        F_train = F_train.add(store_means, axis=1)
        F_train["Date"] = train_dept_ts["Date"]

        smooth_dept_train = pd.melt(F_train, id_vars=['Date'], value_vars = stores_list, \
                                    var_name='Store', value_name='Weekly_Sales')
        smooth_dept_train["Dept"] = department
        smooth_dept_train["Store"] = smooth_dept_train["Store"].astype(np.int64)
        smooth_dept_trains.append(smooth_dept_train)

    smooth_train = pd.concat(smooth_dept_trains, ignore_index=True)
    return smooth_train

def pca_smooth_train(train):
    return pca_smooth_train_v2(train)

In [576]:
%%time
# The code is for fold number =2

foldnum = 2

for foldnum in tqdm(range(1, 11)):
#for foldnum in tqdm([1]):
    # Reading train data
    file_path = f'Proj2_Data/fold_{foldnum}/train.csv'
    train = pd.read_csv(file_path)
    
#     print(f"train=\n {train}")
    
    # Smooth train data
    smoothed = pca_smooth_train(train)
    train_dupe = train[["Date", "IsHoliday"]].drop_duplicates()
    train = smoothed.merge(train_dupe, on=["Date"], how="left")

    # Reading test data
    file_path = f'Proj2_Data/fold_{foldnum}/test.csv'
    test = pd.read_csv(file_path)

    # pre-allocate a pd to store the predictions
    test_pred = pd.DataFrame()

    train_pairs = train[['Store', 'Dept']].drop_duplicates(ignore_index=True)
    test_pairs = test[['Store', 'Dept']].drop_duplicates(ignore_index=True)
    unique_pairs = pd.merge(train_pairs, test_pairs, how = 'inner', on =['Store', 'Dept'])
    #unique_pairs = pd.merge(train_pairs, test_pairs, how = 'outer', on =['Store', 'Dept'])
#     print(f"train_pairs={train_pairs}")
#     print(f"test_pairs={test_pairs}")
#     print(f"unique_pairs={unique_pairs}")

    train_split = unique_pairs.merge(train, on=['Store', 'Dept'], how='left')
#     print(f"train_split=\n{train_split}")
    train_split = preprocess(train_split)
#     print(f"train_split=\n{train_split}")
    y, X = patsy.dmatrices('Weekly_Sales ~ Weekly_Sales + Store + Dept + Yr  + Wk', 
                           data = train_split, 
                           return_type='dataframe')
    train_split = dict(tuple(X.groupby(['Store', 'Dept'])))
#     print(f"y=\n{y}")
#     print(f"X=\n{X}")


    test_split = unique_pairs.merge(test, on=['Store', 'Dept'], how='left')
    test_split = preprocess(test_split)
    y, X = patsy.dmatrices('Yr ~ Store + Dept + Yr  + Wk', 
                           data = test_split, 
                           return_type='dataframe')
    X['Date'] = test_split['Date']
    test_split = dict(tuple(X.groupby(['Store', 'Dept'])))

    keys = list(train_split)
#     print(f"keys=\n{keys}")

    for key in keys:
        X_train = train_split[key]
        X_test = test_split[key]

        Y = X_train['Weekly_Sales']
        X_train = X_train.drop(['Weekly_Sales','Store', 'Dept'], axis=1)

        cols_to_drop = X_train.columns[(X_train == 0).all()]
        X_train = X_train.drop(columns=cols_to_drop)
        X_test = X_test.drop(columns=cols_to_drop)

        cols_to_drop = []
        for i in range(len(X_train.columns) - 1, 1, -1):  # Start from the last column and move backward
            col_name = X_train.columns[i]
            # Extract the current column and all previous columns
            tmp_Y = X_train.iloc[:, i].values
            tmp_X = X_train.iloc[:, :i].values

            coefficients, residuals, rank, s = np.linalg.lstsq(tmp_X, tmp_Y, rcond=None)
            if np.sum(residuals) < 1e-10:
                    cols_to_drop.append(col_name)

        X_train = X_train.drop(columns=cols_to_drop)
        X_test = X_test.drop(columns=cols_to_drop)

        model = sm.OLS(Y, X_train).fit()
        mycoef = model.params.fillna(0)

        tmp_pred = X_test[['Store', 'Dept', 'Date']]
        X_test = X_test.drop(['Store', 'Dept', 'Date'], axis=1)

        tmp_pred['Weekly_Pred'] = np.dot(X_test, mycoef)
        test_pred = pd.concat([test_pred, tmp_pred], ignore_index=True)

    test_pred['Weekly_Pred'].fillna(0, inplace=True)

    file_path = f'Proj2_Data/fold_{foldnum}/mypred.csv'
    print(file_path)
    test_pred.to_csv(file_path, index=False)

 10%|███████████▊                                                                                                          | 1/10 [00:25<03:52, 25.82s/it]

Proj2_Data/fold_1/mypred.csv


 20%|███████████████████████▌                                                                                              | 2/10 [00:54<03:39, 27.43s/it]

Proj2_Data/fold_2/mypred.csv


 30%|███████████████████████████████████▍                                                                                  | 3/10 [01:23<03:16, 28.12s/it]

...

Proj2_Data/fold_8/mypred.csv


 90%|██████████████████████████████████████████████████████████████████████████████████████████████████████████▏           | 9/10 [06:03<00:46, 46.99s/it]

Proj2_Data/fold_9/mypred.csv


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [06:40<00:00, 40.03s/it]

Proj2_Data/fold_10/mypred.csv
CPU times: user 33min 23s, sys: 20min 16s, total: 53min 40s
Wall time: 6min 40s


In [577]:
%%time

wae = myeval()


wae_by_fold=1945.900
wae_by_fold=1364.163
wae_by_fold=1384.109
wae_by_fold=1528.781
wae_by_fold=2320.452
wae_by_fold=1638.096
wae_by_fold=1684.565
wae_by_fold=1400.063
wae_by_fold=1418.869
wae_by_fold=1426.822
overall wae=1611.182
CPU times: user 798 ms, sys: 442 ms, total: 1.24 s
Wall time: 458 ms


In [585]:
%%time
# The code is for fold number =2

foldnum = 2

for foldnum in tqdm(range(1, 11)):
#for foldnum in tqdm([1]):
    # Reading train data
    file_path = f'Proj2_Data/fold_{foldnum}/train.csv'
    train = pd.read_csv(file_path)
    
#     print(f"train=\n {train}")
    
    # Smooth train data
    smoothed = pca_smooth_train(train)
    train_dupe = train[["Date", "IsHoliday"]].drop_duplicates()
    train = smoothed.merge(train_dupe, on=["Date"], how="left")

    # Reading test data
    file_path = f'Proj2_Data/fold_{foldnum}/test.csv'
    test = pd.read_csv(file_path)

    # pre-allocate a pd to store the predictions
    test_pred = pd.DataFrame()

    train_pairs = train[['Store', 'Dept']].drop_duplicates(ignore_index=True)
    test_pairs = test[['Store', 'Dept']].drop_duplicates(ignore_index=True)
    unique_pairs = pd.merge(train_pairs, test_pairs, how = 'inner', on =['Store', 'Dept'])
    #unique_pairs = pd.merge(train_pairs, test_pairs, how = 'outer', on =['Store', 'Dept'])
#     print(f"train_pairs={train_pairs}")
#     print(f"test_pairs={test_pairs}")
#     print(f"unique_pairs={unique_pairs}")

    train_split = unique_pairs.merge(train, on=['Store', 'Dept'], how='left')
#     print(f"train_split=\n{train_split}")
    train_split = preprocess(train_split)
#     print(f"train_split=\n{train_split}")
    y, X = patsy.dmatrices('Weekly_Sales ~ Weekly_Sales + Store + Dept + Yr  + np.power(Yr, 2)+ Wk', 
                           data = train_split, 
                           return_type='dataframe')
    train_split = dict(tuple(X.groupby(['Store', 'Dept'])))
#     print(f"y=\n{y}")
#     print(f"X=\n{X}")


    test_split = unique_pairs.merge(test, on=['Store', 'Dept'], how='left')
    test_split = preprocess(test_split)
    y, X = patsy.dmatrices('Yr ~ Store + Dept + Yr  + np.power(Yr, 2) + Wk', 
                           data = test_split, 
                           return_type='dataframe')
    X['Date'] = test_split['Date']
    test_split = dict(tuple(X.groupby(['Store', 'Dept'])))

    keys = list(train_split)
#     print(f"keys=\n{keys}")

    for key in keys:
        X_train = train_split[key]
        X_test = test_split[key]

        Y = X_train['Weekly_Sales']
        X_train = X_train.drop(['Weekly_Sales','Store', 'Dept'], axis=1)

        cols_to_drop = X_train.columns[(X_train == 0).all()]
        X_train = X_train.drop(columns=cols_to_drop)
        X_test = X_test.drop(columns=cols_to_drop)

        cols_to_drop = []
        for i in range(len(X_train.columns) - 1, 1, -1):  # Start from the last column and move backward
            col_name = X_train.columns[i]
            # Extract the current column and all previous columns
            tmp_Y = X_train.iloc[:, i].values
            tmp_X = X_train.iloc[:, :i].values

            coefficients, residuals, rank, s = np.linalg.lstsq(tmp_X, tmp_Y, rcond=None)
            if np.sum(residuals) < 1e-10:
                    cols_to_drop.append(col_name)

        X_train = X_train.drop(columns=cols_to_drop)
        X_test = X_test.drop(columns=cols_to_drop)

        model = sm.OLS(Y, X_train).fit()
        mycoef = model.params.fillna(0)

        tmp_pred = X_test[['Store', 'Dept', 'Date']]
        X_test = X_test.drop(['Store', 'Dept', 'Date'], axis=1)

        tmp_pred['Weekly_Pred'] = np.dot(X_test, mycoef)
        test_pred = pd.concat([test_pred, tmp_pred], ignore_index=True)

    test_pred['Weekly_Pred'].fillna(0, inplace=True)

    file_path = f'Proj2_Data/fold_{foldnum}/mypred.csv'
    print(file_path)
    test_pred.to_csv(file_path, index=False)

 10%|███████████▊                                                                                                          | 1/10 [00:29<04:29, 29.92s/it]

Proj2_Data/fold_1/mypred.csv


 20%|███████████████████████▌                                                                                              | 2/10 [00:59<03:59, 29.96s/it]

Proj2_Data/fold_2/mypred.csv


 30%|███████████████████████████████████▍                                                                                  | 3/10 [01:33<03:42, 31.77s/it]

...

Proj2_Data/fold_8/mypred.csv


 90%|██████████████████████████████████████████████████████████████████████████████████████████████████████████▏           | 9/10 [06:11<00:49, 49.06s/it]

Proj2_Data/fold_9/mypred.csv


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [07:05<00:00, 42.56s/it]

Proj2_Data/fold_10/mypred.csv
CPU times: user 37min 51s, sys: 21min 10s, total: 59min 1s
Wall time: 7min 5s


In [587]:
%%time

wae = myeval()


wae_by_fold=1945.900
wae_by_fold=1364.163
wae_by_fold=1384.109
wae_by_fold=1528.781
wae_by_fold=2320.452
wae_by_fold=1638.096
wae_by_fold=1614.847
wae_by_fold=1355.272
wae_by_fold=1337.454
wae_by_fold=1334.495
overall wae=1582.357
CPU times: user 482 ms, sys: 241 ms, total: 723 ms
Wall time: 631 ms


In [588]:
%%time

num_folds = 10

file_path = "Proj2_Data/test_with_label.csv"
test_with_label = pd.read_csv(file_path)

for j in range(1, num_folds + 1):
    start_time = time.time()

    # Reading train data
    file_path = f"Proj2_Data/fold_{j}/train.csv"
    train = pd.read_csv(file_path)

    # Smooth train data
    smoothed = pca_smooth_train(train)

    train_dupe = train[["Date", "IsHoliday"]].drop_duplicates()
    train = smoothed.merge(train_dupe, on=["Date"], how="left")

    # Reading test data
    file_path = f"Proj2_Data/fold_{j}/test.csv"
    test = pd.read_csv(file_path)

    # pre-allocate a pd to store the predictions
    test_pred = pd.DataFrame()

    train_pairs = train[["Store", "Dept"]].drop_duplicates(ignore_index=True)
    test_pairs = test[["Store", "Dept"]].drop_duplicates(ignore_index=True)
    unique_pairs = pd.merge(
        train_pairs, test_pairs, how="inner", on=["Store", "Dept"]
    )

    train_split = unique_pairs.merge(train, on=["Store", "Dept"], how="left")
    train_split = preprocess(train_split)
    y, X = patsy.dmatrices(
        "Weekly_Sales ~ Weekly_Sales + Store + Dept + Yr + np.power(Yr, 2)  + Wk",
        data=train_split,
        return_type="dataframe",
    )
    train_split = dict(tuple(X.groupby(["Store", "Dept"])))

    test_split = unique_pairs.merge(test, on=["Store", "Dept"], how="left")
    test_split = preprocess(test_split)
    y, X = patsy.dmatrices(
        "Yr ~ Store + Dept + Yr + np.power(Yr, 2) + Wk", data=test_split, return_type="dataframe"
    )
    X["Date"] = test_split["Date"]
    test_split = dict(tuple(X.groupby(["Store", "Dept"])))

    keys = list(train_split)

    for key in keys:
        X_train = train_split[key]
        X_test = test_split[key]

        Y = X_train["Weekly_Sales"]
        X_train = X_train.drop(["Weekly_Sales", "Store", "Dept"], axis=1)

        model = sm.OLS(Y, X_train).fit()
        mycoef = model.params.fillna(0)

        tmp_pred = X_test[["Store", "Dept", "Date"]]
        X_test = X_test.drop(["Store", "Dept", "Date"], axis=1)

        tmp_pred["Weekly_Pred"] = np.dot(X_test, mycoef)
        test_pred = pd.concat([test_pred, tmp_pred], ignore_index=True)

    test_pred["Weekly_Pred"].fillna(0, inplace=True)

    # Post-prediction adjustment for fold 5
    if j == 5:
        dates = pd.to_datetime(test_pred["Date"])
        test_pred["Wk"] = dates.dt.isocalendar().week

        test_pred_51 = test_pred[test_pred["Wk"] == 51]
        test_pred_51["Shift"] = test_pred_51["Weekly_Pred"] / 9
        test_pred_52 = test_pred[test_pred["Wk"] == 52]

        test_pred_52 = test_pred_52.merge(
            test_pred_51[["Store", "Dept", "Shift"]],
            on=["Store", "Dept"], how="left"
        )

        test_pred_51 = test_pred_51.merge(
            test_pred_52[["Store", "Dept"]],
            on=["Store", "Dept"], how="left", indicator=True
        )
        test_pred_51[test_pred_51["_merge"] == "left_only"]["Shift"] = 0

        test_pred_52["Date"] = "2011-12-30"
        test_pred_52["Shift"].fillna(0, inplace=True)
        test_pred_52["Weekly_Pred"].fillna(0, inplace=True)

        # test_pred_51["Weekly_Pred"] = test_pred_51["Weekly_Pred"] - test_pred_51["Shift"]
        test_pred_52["Weekly_Pred"] = test_pred_52["Weekly_Pred"] + test_pred_52["Shift"]

        test_pred_51.drop("Shift", inplace=True, axis=1)
        test_pred_52.drop("Shift", inplace=True, axis=1)

        test_pred = test_pred[(test_pred["Wk"] != 51) & (test_pred["Wk"] != 52)]
        test_pred = pd.concat([test_pred, test_pred_51, test_pred_52], ignore_index=True)
        test_pred.drop(columns=["Wk"], inplace=True)

    # Save the output to CSV
    file_path = f"Proj2_Data/fold_{j}/mypred.csv"
    test_pred.to_csv(file_path, index=False)

    print(
        "--- fold {fold}, {wae_fold}, {elapsed} seconds ---".format(
            fold=j,
            wae_fold=f"{myeval_fold(j, test_with_label):.3f}",
            elapsed=time.time() - start_time
        )
    )


--- fold 1, 1945.895, 8.539409875869751 seconds ---
--- fold 2, 1364.090, 10.020219087600708 seconds ---
--- fold 3, 1384.009, 10.18326711654663 seconds ---
--- fold 4, 1528.647, 11.091276168823242 seconds ---
--- fold 5, 2220.198, 13.665432691574097 seconds ---
--- fold 6, 1636.200, 20.71449303627014 seconds ---
--- fold 7, 1614.822, 13.318205833435059 seconds ---
--- fold 8, 1355.266, 13.36004090309143 seconds ---
--- fold 9, 1337.461, 12.044707775115967 seconds ---
--- fold 10, 1334.495, 12.105077028274536 seconds ---
CPU times: user 11min 12s, sys: 5min 13s, total: 16min 26s
Wall time: 2min 5s


In [589]:
%%time

wae = myeval()


wae_by_fold=1945.895
wae_by_fold=1364.090
wae_by_fold=1384.009
wae_by_fold=1528.647
wae_by_fold=2220.198
wae_by_fold=1636.200
wae_by_fold=1614.822
wae_by_fold=1355.266
wae_by_fold=1337.461
wae_by_fold=1334.495
overall wae=1572.108
CPU times: user 680 ms, sys: 428 ms, total: 1.11 s
Wall time: 466 ms
